In [3]:
import pandas as pd

# 1. Load the data
file_path = "Credit Card Delinquency Watch.xlsx"  # make sure this file is in the same folder as the notebook
df = pd.read_excel("../data/Credit Card Delinquency Watch.xlsx", "Sample")

# 2. Basic info
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())

print("\nData Types:")
print(df.dtypes)

print("\nFirst 5 rows:")
print(df.head())


Shape: (100, 9)

Columns: ['Customer ID', 'Credit Limit', 'Utilisation %', 'Avg Payment Ratio', 'Min Due Paid Frequency', 'Merchant Mix Index', 'Cash Withdrawal %', 'Recent Spend Change %', 'DPD Bucket Next Month']

Data Types:
Customer ID                object
Credit Limit                int64
Utilisation %               int64
Avg Payment Ratio           int64
Min Due Paid Frequency      int64
Merchant Mix Index        float64
Cash Withdrawal %           int64
Recent Spend Change %       int64
DPD Bucket Next Month       int64
dtype: object

First 5 rows:
  Customer ID  Credit Limit  Utilisation %  Avg Payment Ratio  \
0        C001        165000             12                 32   
1        C002         95000             10                 49   
2        C003         60000             14                 88   
3        C004        125000             99                 65   
4        C005        115000             23                 48   

   Min Due Paid Frequency  Merchant Mix Index 

In [4]:
# Create Delinquent Flag (Target)
df['Delinquent_Flag'] = (df['DPD Bucket Next Month'] >= 1).astype(int)

# Check distribution
print(df['Delinquent_Flag'].value_counts())
print("\nPercentage distribution:")
print(df['Delinquent_Flag'].value_counts(normalize=True) * 100)

# View few rows
df[['Customer ID', 'DPD Bucket Next Month', 'Delinquent_Flag']].head()


Delinquent_Flag
0    79
1    21
Name: count, dtype: int64

Percentage distribution:
Delinquent_Flag
0    79.0
1    21.0
Name: proportion, dtype: float64


,Customer ID,DPD Bucket Next Month,Delinquent_Flag
0,C001,3,1
1,C002,0,0
2,C003,0,0
3,C004,0,0
4,C005,0,0


In [6]:
# F1: Customer usually pays less than half of their credit card bill
df['F1_PaymentLow'] = (df['Avg Payment Ratio'] < 50).astype(int)
df[['Customer ID', 'Avg Payment Ratio', 'F1_PaymentLow']].head()


,Customer ID,Avg Payment Ratio,F1_PaymentLow
0,C001,32,1
1,C002,49,1
2,C003,88,0
3,C004,65,0
4,C005,48,1


In [7]:
#  Flag 2: Spend Drop
df['F2_SpendDrop'] = (df['Recent Spend Change %'] < -15).astype(int)
df[['Customer ID', 'Recent Spend Change %', 'F1_PaymentLow', 'F2_SpendDrop']].head()


,Customer ID,Recent Spend Change %,F1_PaymentLow,F2_SpendDrop
0,C001,-21,1,1
1,C002,1,1,0
2,C003,-15,0,0
3,C004,-23,0,1
4,C005,7,1,0


In [8]:
# Flag 3: Merchant Mix Index
# It reflects what type of merchants the customer is spending on.
# Higher index → more spending at high-risk categories (like clubs, betting, gambling, discretionary purchases)
# Lower index → spending on basic needs (grocery, bill payments etc.)
 # Typically, banks consider >0.70 as risky.
# F3: High Merchant Risk
df['F3_HighMerchantRisk'] = (df['Merchant Mix Index'] > 0.70).astype(int)

df[['Customer ID', 'Merchant Mix Index', 'F3_HighMerchantRisk']].head()


,Customer ID,Merchant Mix Index,F3_HighMerchantRisk
0,C001,0.73,1
1,C002,0.42,0
2,C003,0.96,1
3,C004,0.79,1
4,C005,0.64,0


In [9]:
# Flag 4: High Utilisation (Conditional)
# If a customer is using more than 75% of their credit limit, that’s risky because they are maxing out the card.
# BUT not always risky unless paired with low payment capacity or spend drop.
#  So we make this conditional, more intelligent:
# F4: High Utilisation (only risky if payment ratio low or spending dropped)
df['F4_HighUtilisation'] = (
    (df['Utilisation %'] > 75) &
    ((df['F1_PaymentLow'] == 1) | (df['F2_SpendDrop'] == 1))
).astype(int)

df[['Customer ID', 'Utilisation %', 'F1_PaymentLow', 'F2_SpendDrop', 'F4_HighUtilisation']].head()


,Customer ID,Utilisation %,F1_PaymentLow,F2_SpendDrop,F4_HighUtilisation
0,C001,12,1,1,0
1,C002,10,1,0,0
2,C003,14,0,0,0
3,C004,99,0,1,1
4,C005,23,1,0,0


In [10]:
# F5: Payment Stress
# If customer repeatedly pays only minimum due (i.e., >50% of months) AND payment ratio is low

df['F5_PaymentStress'] = (
    (df['Avg Payment Ratio'] < 50) &
    (df['Min Due Paid Frequency'] > 50)
).astype(int)

df[['Customer ID', 'Avg Payment Ratio', 'Min Due Paid Frequency', 'F5_PaymentStress']].head()


,Customer ID,Avg Payment Ratio,Min Due Paid Frequency,F5_PaymentStress
0,C001,32,66,1
1,C002,49,45,0
2,C003,88,23,0
3,C004,65,31,0
4,C005,48,46,0


In [11]:
# F6: Cash Withdrawal Risk
df['F6_CashWithdrawRisk'] = (
    (df['Cash Withdrawal %'] > 20) &
    (df['Avg Payment Ratio'] < 50)
).astype(int)

df[['Customer ID', 'Cash Withdrawal %', 'Avg Payment Ratio', 'F6_CashWithdrawRisk']].head()


,Customer ID,Cash Withdrawal %,Avg Payment Ratio,F6_CashWithdrawRisk
0,C001,12,32,0
1,C002,20,49,0
2,C003,9,88,0
3,C004,6,65,0
4,C005,13,48,0


In [12]:
# List of all flags
flag_columns = [
    'F1_PaymentLow', 'F2_SpendDrop', 'F3_HighMerchantRisk',
    'F4_HighUtilisation', 'F5_PaymentStress', 'F6_CashWithdrawRisk'
]

# Total risk flags count per customer
df['Total_Risk_Flags'] = df[flag_columns].sum(axis=1)

# Risk level classification
df['Risk_Level'] = df['Total_Risk_Flags'].apply(
    lambda x: 'High' if x >= 3 else ('Medium' if x == 2 else 'Low')
)

# Display first few rows
df[['Customer ID', 'Total_Risk_Flags', 'Risk_Level'] + flag_columns].head()


,Customer ID,Total_Risk_Flags,Risk_Level,F1_PaymentLow,F2_SpendDrop,F3_HighMerchantRisk,F4_HighUtilisation,F5_PaymentStress,F6_CashWithdrawRisk
0,C001,4,High,1,1,1,0,1,0
1,C002,1,Low,1,0,0,0,0,0
2,C003,1,Low,0,0,1,0,0,0
3,C004,3,High,0,1,1,1,0,0
4,C005,1,Low,1,0,0,0,0,0


In [13]:
# Risk Summary Distribution
risk_summary = df.groupby('Risk_Level')['Customer ID'].count().reset_index()
risk_summary.columns = ['Risk_Level', 'Customer_Count']
risk_summary['Percentage'] = (risk_summary['Customer_Count'] / len(df) * 100).round(2)

# Compare delinquency rate by risk category
delinquency_summary = df.groupby('Risk_Level')['Delinquent_Flag'].mean().reset_index()
delinquency_summary.columns = ['Risk_Level', 'Actual_Delinquency_Rate (%)']
delinquency_summary['Actual_Delinquency_Rate (%)'] = (delinquency_summary['Actual_Delinquency_Rate (%)'] * 100).round(2)

# Merge both reports
final_summary = risk_summary.merge(delinquency_summary, on='Risk_Level')
final_summary


,Risk_Level,Customer_Count,Percentage,Actual_Delinquency_Rate (%)
0,High,17,17.0,29.41
1,Low,65,65.0,16.92
2,Medium,18,18.0,27.78


In [14]:
# Save output to Excel
df.to_excel("Early_Risk_Signal_Output.xlsx", index=False)
print("✔ File saved successfully: Early_Risk_Signal_Output.xlsx")


✔ File saved successfully: Early_Risk_Signal_Output.xlsx


In [ ]:
# 1) Payment_Stress_Score
df['Payment_Stress_Score'] = (
    (100 - df['Avg Payment Ratio']) * (df['Min Due Paid Frequency'] / 100)
)

# 2) Behaviour_Risk_Score
df['Behaviour_Risk_Score'] = (
    0.3 * df['Utilisation %'] +
    0.3 * (100 - df['Avg Payment Ratio']) +
    0.2 * df['Min Due Paid Frequency'] +
    0.1 * df['Cash Withdrawal %'] +
    0.1 * (df['Merchant Mix Index'] * 100)
)

# View first few rows for these new features
df[['Customer ID',
    'Avg Payment Ratio',
    'Min Due Paid Frequency',
    'Payment_Stress_Score',
    'Behaviour_Risk_Score']].head()


Adding new features to the dataset

In [15]:
# 1) Payment_Stress_Score
df['Payment_Stress_Score'] = (
    (100 - df['Avg Payment Ratio']) * (df['Min Due Paid Frequency'] / 100)
)

# 2) Behaviour_Risk_Score
df['Behaviour_Risk_Score'] = (
    0.3 * df['Utilisation %'] +
    0.3 * (100 - df['Avg Payment Ratio']) +
    0.2 * df['Min Due Paid Frequency'] +
    0.1 * df['Cash Withdrawal %'] +
    0.1 * (df['Merchant Mix Index'] * 100)
)

# View first few rows for these new features
df[['Customer ID',
    'Avg Payment Ratio',
    'Min Due Paid Frequency',
    'Payment_Stress_Score',
    'Behaviour_Risk_Score']].head()


,Customer ID,Avg Payment Ratio,Min Due Paid Frequency,Payment_Stress_Score,Behaviour_Risk_Score
0,C001,32,66,44.88,45.7
1,C002,49,45,22.95,33.5
2,C003,88,23,2.76,22.9
3,C004,65,31,10.85,54.9
4,C005,48,46,23.92,39.4


In [17]:
# Adding Risk_Reasons_Text
def get_risk_reasons(row):
    reasons = []
    
    if row['F1_PaymentLow'] == 1:
        reasons.append("Low Payment Ratio")
    if row['F2_SpendDrop'] == 1:
        reasons.append("Recent Spend Drop")
    if row['F3_HighMerchantRisk'] == 1:
        reasons.append("High-Risk Merchant Spending")
    if row['F4_HighUtilisation'] == 1:
        reasons.append("High Utilisation")
    if row['F5_PaymentStress'] == 1:
        reasons.append("Payment Stress")
    if row['F6_CashWithdrawRisk'] == 1:
        reasons.append("High Cash Withdrawal")
    
    # If no flags triggered
    return ", ".join(reasons) if reasons else "Stable behaviour"


# Apply function
df['Risk_Reasons_Text'] = df.apply(get_risk_reasons, axis=1)

# Show results
df[['Customer ID', 'Risk_Level', 'Total_Risk_Flags', 'Risk_Reasons_Text']].head()


,Customer ID,Risk_Level,Total_Risk_Flags,Risk_Reasons_Text
0,C001,High,4,"Low Payment Ratio, Recent Spend Drop, High-Ris..."
1,C002,Low,1,Low Payment Ratio
2,C003,Low,1,High-Risk Merchant Spending
3,C004,High,3,"Recent Spend Drop, High-Risk Merchant Spending..."
4,C005,Low,1,Low Payment Ratio


In [18]:
# Check score distribution
df[['Behaviour_Risk_Score', 'Payment_Stress_Score', 'Risk_Level']].groupby('Risk_Level').mean().round(2)


,Behaviour_Risk_Score,Payment_Stress_Score
Risk_Level,,
High,52.32,22.68
Low,40.01,14.81
Medium,49.24,20.95


In [19]:
df['Behaviour_Risk_Category'] = df['Behaviour_Risk_Score'].apply(
    lambda x: 'High' if x >= 50 else ('Medium' if x >= 45 else 'Low')
)

df['Payment_Stress_Category'] = df['Payment_Stress_Score'].apply(
    lambda x: 'High' if x >= 25 else ('Medium' if x >= 15 else 'Low')
)

df[['Customer ID', 'Risk_Level', 'Behaviour_Risk_Score', 'Behaviour_Risk_Category', 'Payment_Stress_Score', 'Payment_Stress_Category']].head()


,Customer ID,Risk_Level,Behaviour_Risk_Score,Behaviour_Risk_Category,Payment_Stress_Score,Payment_Stress_Category
0,C001,High,45.7,Medium,44.88,High
1,C002,Low,33.5,Low,22.95,Medium
2,C003,Low,22.9,Low,2.76,Low
3,C004,High,54.9,High,10.85,Low
4,C005,Low,39.4,Low,23.92,Medium


In [20]:
# wrapping everything in a function
def run_risk_engine(df_input: pd.DataFrame) -> pd.DataFrame:
    """
    Take a dataframe with the original customer behaviour columns
    and return a new dataframe with:
    - Risk flags (F1..F6)
    - Total_Risk_Flags
    - Risk_Level
    - Engineered scores and categories
    - Risk_Reasons_Text
    """
    df = df_input.copy()

    # Target (only for analysis, not needed for future prediction use)
    if 'DPD Bucket Next Month' in df.columns:
        df['Delinquent_Flag'] = (df['DPD Bucket Next Month'] >= 1).astype(int)

    # F1–F6 flags
    df['F1_PaymentLow'] = (df['Avg Payment Ratio'] < 50).astype(int)
    df['F2_SpendDrop'] = (df['Recent Spend Change %'] < -15).astype(int)
    df['F3_HighMerchantRisk'] = (df['Merchant Mix Index'] > 0.70).astype(int)

    df['F4_HighUtilisation'] = (
        (df['Utilisation %'] > 75) &
        ((df['F1_PaymentLow'] == 1) | (df['F2_SpendDrop'] == 1))
    ).astype(int)

    df['F5_PaymentStress'] = (
        (df['Avg Payment Ratio'] < 50) &
        (df['Min Due Paid Frequency'] > 50)
    ).astype(int)

    df['F6_CashWithdrawRisk'] = (
        (df['Cash Withdrawal %'] > 20) &
        (df['Avg Payment Ratio'] < 50)
    ).astype(int)

    flag_columns = [
        'F1_PaymentLow', 'F2_SpendDrop', 'F3_HighMerchantRisk',
        'F4_HighUtilisation', 'F5_PaymentStress', 'F6_CashWithdrawRisk'
    ]

    df['Total_Risk_Flags'] = df[flag_columns].sum(axis=1)

    # Risk level from flags
    df['Risk_Level'] = df['Total_Risk_Flags'].apply(
        lambda x: 'High' if x >= 3 else ('Medium' if x == 2 else 'Low')
    )

    # Engineered scores
    df['Payment_Stress_Score'] = (
        (100 - df['Avg Payment Ratio']) * (df['Min Due Paid Frequency'] / 100)
    )

    df['Behaviour_Risk_Score'] = (
        0.3 * df['Utilisation %'] +
        0.3 * (100 - df['Avg Payment Ratio']) +
        0.2 * df['Min Due Paid Frequency'] +
        0.1 * df['Cash Withdrawal %'] +
        0.1 * (df['Merchant Mix Index'] * 100)
    )

    # Score categories
    df['Behaviour_Risk_Category'] = df['Behaviour_Risk_Score'].apply(
        lambda x: 'High' if x >= 50 else ('Medium' if x >= 45 else 'Low')
    )

    df['Payment_Stress_Category'] = df['Payment_Stress_Score'].apply(
        lambda x: 'High' if x >= 25 else ('Medium' if x >= 15 else 'Low')
    )

    # Reasons text
    def get_risk_reasons(row):
        reasons = []
        if row['F1_PaymentLow'] == 1:
            reasons.append("Low Payment Ratio")
        if row['F2_SpendDrop'] == 1:
            reasons.append("Recent Spend Drop")
        if row['F3_HighMerchantRisk'] == 1:
            reasons.append("High-Risk Merchant Spending")
        if row['F4_HighUtilisation'] == 1:
            reasons.append("High Utilisation")
        if row['F5_PaymentStress'] == 1:
            reasons.append("Payment Stress")
        if row['F6_CashWithdrawRisk'] == 1:
            reasons.append("High Cash Withdrawal")
        return ", ".join(reasons) if reasons else "Stable behaviour"

    df['Risk_Reasons_Text'] = df.apply(get_risk_reasons, axis=1)

    return df
